# Synopsis

Regression analysis is one of the most common types of analysis of data in science.  

##  Words to remember

**ordinary least squares (OLS) regression**

**intercept**

**linear coefficient**

**residuals**


# Read libraries

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from colorama import Back, Fore, Style
from pathlib import Path
from sys import path

path.append( str(Path.cwd().parent) )
path

In [ ]:
my_fontsize = 15

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.formula.api as smf

from copy import copy
from matplotlib import gridspec
from random import sample, choice, choices

from Amaral_libraries.my_stats import half_frame


# Linear Regression

Science and engineering are about obtaining associations between multiple variables. For example, the ideal gas law relates several quantities that can be measured for a gas sample:

> $ PV = n R T$.

For constant volume and number of moles, the equation becomes a simple linear relationship between pressure and temperature

> $ P = a T$

You can imagine a situation in which some 16th Century experimental physicist was trying to determine whether there is an association between $P$ and $T$ and making noisy measurements of these two quantities.

Regression analysis is a statistical approach that is used to investigate for the existence of associations such as the ones described above. The basic assumption of **linear** regression analysis is that 

> $y_i = \beta_0 + \beta_1 x_i + \epsilon_i$ . 

So, 

> $\epsilon_i = y_i - \beta_0 - \beta_1 x_i$

where the $\{\epsilon_i\}$ are assumed to be i.i.d. normal variables with zero mean and standard deviation $\sigma$.

If we fix the value of $X$, then we can write that

> $E[Y|X = x] = \beta_0 + \beta_1 x + E[\epsilon_i] = \beta_0 + \beta_1 x$

and that

> $E\left[(Y - E[Y|X = x])^2 ~| ~X = x \right] = E[\epsilon_i ^2] = \sigma^2$

<br><br><br>



# Least squares estimation

If we have data, $\{x_i\}$ and $\{y_i\}$, and want to estimate $\beta_0$ and $\beta_1$, we can accomplish it by minimizing the squared differences of the data to our linear model.

> $L = \sum^n_{i = 1} \epsilon_i ^2 = \sum^n_{i = 1} \left( y_i - \beta_0 - \beta_1 x_i \right)^2$

The minimization process yields the solutions:

> $\hat \beta_1 = \frac{\bar{xy} - \frac{\bar x ~\bar y}{n}}{\bar{ x^2 } - \frac{\bar x ^2}{n}}$

> $\hat \beta_0 = \bar y - \beta_1 \bar x$

And $e_i = y_i - \hat \beta_0 - \hat \beta_1 x_i$ are the **residuals of the fit of the linear model to the data**.  

In [ ]:
# Set parameters and define noises
n = 20
beta_0, beta_1 = -1.0, 1.0
sigma_y = 0.20

# Set to zero if no undertainty in x measurement
# sigma_x = sigma_y / 10
sigma_x = 0.0

# Generate noise terms for y and x variables
x_noise = stats.norm.rvs(0, sigma_x, n+1)
y_noise = stats.norm.rvs(0, sigma_y, n+1)

# Create noisy measurements
#
x = np.linspace(0, 1., n+1)
y = beta_0 + beta_1 * x + y_noise
x = x + x_noise

# Place data into dataframe
#
df = pd.DataFrame(list(zip(x, y)), columns=['X', 'Y'])

# This is an R-style linear model construction
model = smf.ols(formula = 'Y ~ X ', data = df)

results = model.fit()
print(f"results is a {type(results)}\n")
print(f"results.params is a {type(results.params)}\n")
print(f"{results.params}\n\n")

print("We can access them using the .column formalism.")
print(f"The intercept is {results.params.Intercept:.3f}")
print(f"The coefficient for X is {results.params.X:.3f}")

In [ ]:
print(f"results.summary() is a {type(results.summary())}\n\n")
print(results.summary())


<br><br><br>

## The values on the right side of the top section deal with the quality of the fit

The **$R^2$** statistic measures what proportion of the variation in the outcome Y that can be explained by the covariates/predictors. If $R^2$ is close to 1, it means that the covariates can jointly fully explain the variation in the outcome Y. This means Y can be accurately predicted (in some sense) using the covariates. Conversely, a low $R^2$ means Y is poorly predicted by the covariates. 

The point estimator for $R^2$ is biased. The magnitude of the bias will depend on how many observations are available to fit the model and how many covariates there are relative to this sample size. **The adjusted $R^2$** attempts to correct for the bias.

The `.fit` method uses a statistical test involving the **F-statistic** to help the user decide whether a linear model fits the data significantly better than a null model that assumes just a constant term.

If the **p-value** for the calculated **F-statistic** is smaller than a chosen threshold, one typically uses it to reject the null hypothesis.

There is a saying that *all models are wrong but some are useful*.  There is another saying that *with 2 parameters one can fit a line but that with 5 one can fit an elephant*.

Being able to reject the null hypothesis does not mean that the linear model is correct (we likely have too little data).  Having a more complex model that appears to explain the data better (larger $R^2$) does not mean that it is a better model.

In order to try to balance model complexity and model ability to explain the data, one makes use of so-called information criteria (IC). The two most commonly used are the Akike IC and the Bayesian IC. When looking at a single model, the value reported for the **AIC** and the **BIC** are of no use. Only when we are comparing models and those models are related do they became useful.


<br><br><br>

## The values on the bottom section of the summary table concern the properties of the residuals. 

As you recall, the assumption is that the $\epsilon$'s are Gaussian i.i.d. random variables.  **The skewness of Gaussian random variables is 0 and the kurtosis is 3**. 

The **Jarque–Bera** statistic quantifies how close the sample's skewness and kurtosis are to the expected values for Gaussian distributed random variable.  The test statistic is always **nonnegative**. **If it is far from zero, it signals that the residuals are not normally distributed**. 

The **Durbin-Watson statistic** is used to test for **autocorrelation** of the residuals. The Durbin-Watson statistic will always have a value between 0 and 4. **A value of 2.0 indicates no autocorrelation of the residuals**. Values from 0 to less than 2 indicate positive autocorrelation and values from from 2 to 4 indicate negative autocorrelation.

<br><br><br>

## Visualizing the fit

In [ ]:
fig = plt.figure( figsize = (6, 4.5) )
ax = fig.add_subplot(1,1,1)
half_frame(ax, "$x$", "$y$", font_size = my_fontsize)

# Plot data
#
ax.plot(x, y, 'ro', alpha = 0.9, label = 'data')

# Plot true functional dependence
#
ax.plot(x, beta_0 + x * beta_1,  linewidth = 3, color = 'orange',
          label = f'Generating Model:\n   {beta_0:.2f} + {beta_1:.2f}x')

# Plot linear fit
#
ax.plot( x, results.params[0] + x * results.params[1], 
         '-', color = 'k', linewidth = 2, alpha = 1, 
         label = "OLS fit:  $\\beta_1 = $" + f"{results.params[1]:.3f}" + 
         " $\pm$ " + f"{results.bse[1]:.3f}")

# Plot estimated noise component in data
#
ax.vlines(x, y, results.params[0] + x * results.params[1], color = 'b')

# Format legend
ax.legend(loc = (1.1, 0.5), frameon = False, markerscale = 1.3, 
          fontsize = my_fontsize)

plt.show()

<br><br><br>

The object returned by `model.fit()` enables us to access all values returned by `model.fit().summary()` by calling specific attributed.  For example, point estimates for the coefficients of the linear fit are accessible through the attribute `params` (see label in code cell above).

A list of all attributes and methods can be found [here](https://www.statsmodels.org/0.8.0/generated/statsmodels.regression.linear_model.RegressionResults.html#statsmodels.regression.linear_model.RegressionResults).

The standard errors for the coefficient estimates can be retrieved with the attribute `bse` and the confidence intervals can be retrieved with `conf_int`, which returns a pandas `dataframe`. 

In [ ]:
print(f"results.conf_int() is a {type(results.conf_int())}\n")
results.conf_int()

In [ ]:
print(f"Confidence interval for beta_1 is [{results.conf_int()[0]['X']:.3f}," 
      f" {results.conf_int()[1]['X']:.3f}]" )

In [ ]:
results.conf_int(alpha = 0.01) # Default is alpha = 0.05

# Exploring the impact of different types of association


Consider the case:

> $y = a + b~x~~~~~$ with $b \ll 1$ and $x \in [0,1]$

What do you expect to find? How can you test your hypothesis? Where you correct?


Consider the case:

> $y = a + b~x + c~x^2~~~~~$ with $ b \gg c$ and $x \in [0,1]$

What do you expect to find? How can you test your hypothesis? Where you correct?


Consider the case:

> $y = a + b~x^2~~~~~$ with $x \in [0,1]$

What do you expect to find? How can you test your hypothesis? Where you correct?


Consider the case:

> $\sigma_x \approx \sigma_y$

What do you expect to find? How can you test your hypothesis? Where you correct?
